Fizz Buzz游戏：

给你一个整数n. 从 1 到 n 按照下面的规则打印每个数：
如果这个数被3整除，打印fizz.
如果这个数被5整除，打印buzz.
如果这个数能同时被3和5整除，打印fizz buzz.

先对输入数字进行二值编码。

In [3]:
import numpy as np
import torch

def binary_encoder(input_size):
    def wrapper(num):
        ret = [int(i) for i in '{0:b}'.format(num)]
        return [0] * (input_size - len(ret)) + ret
    return wrapper

def get_numpy_data(input_size, limit=1000):
    x, y = [], []
    encoder = binary_encoder(input_size)
    for i in range(limit):
        x.append(encoder(i))
        if i % 15 == 0:
            y.append([1,0,0,0])
        elif i % 5 == 0:
            y.append([0,1,0,0])
        elif i % 3 == 0:
            y.append([0,0,1,0])
        else:
            y.append([0,0,0,1])
    return np.array(x), np.array(y)

定义超参数

In [4]:
epochs = 500
batches = 64
lr = 0.01
input_size = 10
output_size = 4
hidden_size = 100

自动求导
从get_numpy_data()创建Pytorch张量。

In [11]:
trX, trY = get_numpy_data(input_size)
x = torch.from_numpy(trX)
y = torch.from_numpy(trY)
w1 = torch.randn(input_size, hidden_size, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, requires_grad=True)
b1 = torch.zeros(1, hidden_size, requires_grad=True)
b2 = torch.zeros(1, output_size, requires_grad=True)

In [1]:
import torch

inputs = torch.FloatTensor([2])
weights = torch.rand(1, requires_grad=True)
bias = torch.rand(1, requires_grad=True)
t = inputs @ weights
out = t + bias
out.backward()
weights.grad

tensor([2.])

In [2]:
bias.grad

tensor([1.])

张量实例中主要有三个属性用于支持自动求导：grad， .data, grad_fn().
.grad属性存储任意节点在任意时间点的梯度。.data用于访问包含属性的裸张量对象。
张量实例与Function实例在图上是相互连接的。除了用户明确定义的张量，
每个张量都与一个函数相连。可以在张量上调用grade_fn来访问创建函数。

In [15]:
for epoch in range(epochs):
    for batch in range(10):
        start = batch * batches
        end = start + batches
        x_ = x[start:end]
        y_ = y[start:end]

        a2 = x_.matmul(w1.long())
        a2 = a2.add(b1)
        print(a2.grad, a2.grad_fn, a2)
        print(a2.size())

        h2 = a2.sigmoid()
        a3 = h2.matmul(w2)
        a3 = a3.add(b2)
        hyp = a3.sigmoid()
        error = hyp - y_
        #MSE,均方误差
        output = error.pow(2).sum() / 2.0

        '''w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()'''
        #触发反向传播，并基于梯度更新
        output.backward()

        print(x.grad, x.grad_fn, x)
        print(w1.grad, w1.grad_fn, w1)
        print(a2.grad, a2.grad_fn, a2)

        with torch.no_grad():
            w1 -= lr * w1.grad
            w2 -= lr * w2.grad
            b1 -= lr * b1.grad
            b2 -= b2 * w1.grad

        print(a2.grad, a2.grad_fn, a2)

None <AddBackward0 object at 0x7f48c60490a0> tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        ...,
        [ 2.,  0.,  0.,  ...,  0.,  1., -2.],
        [ 2.,  0.,  0.,  ...,  0.,  2., -2.],
        [ 2.,  0.,  0.,  ...,  0.,  2., -2.]], grad_fn=<AddBackward0>)
torch.Size([64, 100])
None None tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1]])
None None tensor([[-4.8214e-01,  1.0064e+00, -9.6264e-01, -2.1357e+00,  5.0165e-01,
          3.7822e+00, -1.1739e+00, -2.0010e-01,  1.1641e+00, -8.9520e-01,
          1.1721e+00, -9.7261e-01,  1.7953e-02, -1.3551e+00, -1.3430e+00,
          8.7970e-02, -1.2600e-01, -3.3888e-01, -1.3617e+00,  1.5023e+00,
          2.1298e-01,  5.0209e-01, -2.4988e-01,  2.0210e+00, -5.1063e-01,
          1

/tmp/ipykernel_72953/2006177540.py:10: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(a2.grad, a2.grad_fn, a2)
/tmp/ipykernel_72953/2006177540.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(a2.grad, a2.grad_fn, a2)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'